In [4]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import os
import re

%cd ..

d:\DEV\SouJunior\Personal Lab


C:\Users\Moscarde\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:

def inicio(soup_list):
    main_df = pd.read_csv("linkedin_page/dataframes/main_dataframe.csv")
    main_df_post_ids = main_df["linkedin_id"].tolist()

    processando(soup_list, main_df, main_df_post_ids)


def define_date(raw_date, today_date):
    num, mult, _ = raw_date.split(" ")
    if mult == "d":
        publish_date = today_date - timedelta(days=int(num))
    elif mult == "sem":
        publish_date = today_date - timedelta(days=int(num) * 7)
    elif mult == "m":
        publish_date = today_date - timedelta(days=int(num) * 30)
    elif mult == "h":
        publish_date = today_date

    return publish_date.date()

In [27]:
def processando(soup_list, main_df, main_df_post_ids):

    df_new_content = []
    today_date = datetime.now()
    for post in soup_list:
        print()

        publish_date = get_published_date(post, today_date)
        print(publish_date)

        post_url = get_post_url(post)
        print(post_url)

        linkedin_post_id = post_url.split(":")[-1]

        if (int(linkedin_post_id) in main_df_post_ids) or publish_date == None:
            continue

        shared_post = is_shared_post(post)
        print(shared_post)

        author_name = get_author_name(post)
        print(author_name)

        author_profile_url = get_author_profile_url(post)
        print(author_profile_url)

        post_type = get_post_type(post)
        print(post_type)

        post_content = get_post_content(post, post_type)
        print(post_content)

        qtd_reaction = get_qtd_reaction(post)
        print(qtd_reaction)

        qtd_comment = get_qtd_comment(post)
        print(qtd_comment)

        qtd_share = get_qtd_share(post)
        print(qtd_share)

        df_new_content.append(
            [
                publish_date,
                shared_post,
                author_name,
                author_profile_url,
                post_type,
                post_url,
                post_content,
                qtd_reaction,
                qtd_comment,
                qtd_share,
                linkedin_post_id,
            ]
        )


    columns = [
        "publish_date",
        "shared_post",
        "author_name",
        "author_profile_url",
        "post_type",
        "post_url",
        "post_content",
        "qtd_reaction",
        "qtd_comment",
        "qtd_share",
        "linkedin_post_id",
    ]
    df = pd.DataFrame(df_new_content, columns=columns)
    df.sort_values(by="publish_date", ascending=False, inplace=True)
    print(df)

    # #salvando dataframe
    # concat_df = pd.concat([df, main_df], ignore_index=True)
    # print(concat_df)


# concat_df.to_csv("linkedin_page/dataframes/main_dataframe.csv", index=False)
# concat_df.to_csv(f"linkedin_page/dataframes/backup/{today_date.date()}.csv", index=False)

In [20]:
def get_published_date(post, today_date):
    actor_meta = post.find_all(class_="update-components-actor__meta")

    if actor_meta:
        visually_hidden = actor_meta[0].find_all(class_="visually-hidden")

        if visually_hidden:
            last_visually_hidden = visually_hidden[-1].text.split("•")[0]
            return define_date(last_visually_hidden, today_date)

    if (
        post.find(class_="update-components-promo-v1__text-container")
        or len(post.find_all(tag="div")) <= 7
    ):
        return None

    return "Unknown"


def is_shared_post(post):
    if (post.find(class_="update-components-header__text-view") is not None) or (
        "feed-shared-update-v2__update-content-wrapper"
        in post.find(
            class_="feed-shared-update-v2__description-wrapper"
        ).find_next_sibling()["class"]
    ):
        return True
    else:
        return False


def get_author_name(post):
    return post.find(class_="update-components-actor__name").span.span.text


def get_author_profile_url(post):
    return post.find(class_="update-components-actor__meta-link")["href"]


def get_post_type(post):
    if (post.find(class_="update-components-header__text-view") is not None) or (
        "feed-shared-update-v2__update-content-wrapper"
        in post.find(
            class_="feed-shared-update-v2__description-wrapper"
        ).find_next_sibling()["class"]
    ):
        return "Shared Content"

    elif post.find(class_="update-components-poll") is not None:
        return "Poll"

    elif (
        post.find(class_="update-components-scheduled-live-content__event-link")
        is not None
    ):
        return "Scheduled Event"

    elif post.find(class_="video-live") is not None:
        return "Live"

    elif post.find(class_="update-components-image--single-image") is not None:
        return "Single Image"

    elif (
        "update-v2-social-activity"
        in post.find(
            class_="feed-shared-update-v2__description-wrapper"
        ).find_next_sibling()["class"]
    ):
        return "Text Only"

    else:
        return "Unknown type"


def get_post_url(post):
    return (
        "https://www.linkedin.com/feed/update/"
        + post.find(class_="feed-shared-update-v2")["data-urn"]
    )


def get_post_content(post, post_type):
    if post_type == "Shared Content":
        return post.find(
            class_="update-components-update-v2__commentary"
        ).span.span.text
    else:
        return post.find(class_="update-components-text").span.span.text


def get_qtd_reaction(post):
    if post.find(class_="social-details-social-counts__reactions-count") != None:
        return post.find(
            class_="social-details-social-counts__reactions-count"
        ).text.strip()

    else:
        qtd_reactions = post.find(
            class_="social-details-social-counts__social-proof-text"
        ).text.strip()

        re_pattern_reactions = r"\b(\d+)\s+pessoas\b"

        return 1 + int(re.findall(re_pattern_reactions, qtd_reactions)[0])


def get_qtd_comment(post):
    if post.find(class_="social-details-social-counts__comments") != None:
        return (
            post.find(class_="social-details-social-counts__comments")
            .button.span.text.strip()
            .split(" ")[0]
        )
    else:
        return "0"


def get_qtd_share(post):
    if (
        len(post.find_all(class_="social-details-social-counts__item--right-aligned"))
        <= 1
    ):
        return 0
    else:
        return (
            post.find_all(class_="social-details-social-counts__item--right-aligned")[1]
            .button.span.text.strip()
            .split(" ")[0]
        )

In [28]:
#start
staged_data_path = "linkedin_page/staged_data/"
staged_filenames = os.listdir(staged_data_path)
soup_list = []
for filename in staged_filenames:
    with open(staged_data_path + filename, "r", encoding="utf8") as file:
        soup_list.append(BeautifulSoup(file, "html.parser"))

inicio(soup_list[:])


2023-12-13
https://www.linkedin.com/feed/update/urn:li:activity:7140031369270779904

2023-12-14
https://www.linkedin.com/feed/update/urn:li:activity:7142568593040338945
True
marlon jerold
https://www.linkedin.com/in/marlon-jerold?miniProfileUrn=urn%3Ali%3Afsd_profile%3AACoAAC79v8IBwLZ7mbcJfBrNSGNxvxXFkkVqMSE
Shared Content
Salve galera, gostaria de compartilhar essa galera aqui.SouJunior Ces sabem o quanto eu curto quando nós que estamos no início de carreira recebemos oportunidades.Então deem uma olhada nessa galera aí.🍁🦆
52
8
3

2023-12-20
https://www.linkedin.com/feed/update/urn:li:activity:7142645870126137344
True
SouJunior
https://www.linkedin.com/company/soujunior/?miniCompanyUrn=urn%3Ali%3Afsd_company%3A82326952
Shared Content
💙
3
0
0

2023-12-20
https://www.linkedin.com/feed/update/urn:li:activity:7142644442905149441
True
SouJunior
https://www.linkedin.com/company/soujunior/?miniCompanyUrn=urn%3Ali%3Afsd_company%3A82326952
Shared Content
Vagasssss
13
2
1

2023-12-20
https://ww

In [2]:
import pandas as pd

In [19]:
df = pd.read_csv("../linkedin_page/dataframes/backup/main_dataframe_backup_2023-12-28_16-33-38.csv")
df.loc[df["linkedin_post_id"] == 7.145832051659776e+18, "cacto"] = "teste"
df[df["linkedin_post_id"] == 7.145832051659776e+18]

,publish_date,shared_post,author_name,author_profile_url,post_type,post_url,post_content,qtd_reaction,qtd_comment,qtd_share,linkedin_post_id,reaction_count,comment_count,share_count,linkedin_id,cacto
0,2023-12-27,True,SouJunior,https://www.linkedin.com/company/soujunior/?mi...,Shared Content,https://www.linkedin.com/feed/update/urn:li:ac...,🇺🇲,7.0,0.0,0.0,7.145832e+18,NaN,NaN,NaN,NaN,teste
